In [1]:
import sqlite3
import time
import pandas as pd
import numpy as np
import query

from datetime import datetime

종목코드열람

In [2]:
shcode_list = query.query_shcode_list_today()

In [3]:
# array_df: numpy array date, open, high, low, close, volume
def fun_incresing_ratio_test(array_df, limit = 0.15):
    #1) 종가비율 limit 이상
    ratio_list = [y/x-1 for x, y in zip(array_df[:,4][:-1], array_df[:,4][1:])]
    ratio_test = np.array([i >= limit for i in ratio_list])
    ratio_test = np.insert(ratio_test, 0 , 0)
    return ratio_test

In [4]:
def fun_total_volume_test(array_df, min_total_volume = 30000000000):
    
    #2) 해당날 거래대금 300억 이상 거래대금 = 종가 * 거래량
    total_volume_list = array_df[:,4] * array_df[:,5]
    total_volume_test = [i >= min_total_volume  for i in total_volume_list]
    total_volume_test = np.array(total_volume_test[:])
    return total_volume_test


In [5]:
def fun_average_volume_test(array_df, ratio_test, total_volume_test):
    #3) 급등시점 전 평균 20일간 거래량의 1000%
    average_volume_list = ratio_test * total_volume_test
    average_volume_list_index = [ i for i,v in enumerate(average_volume_list) if v == True]
    
    average_volume_test = np.array([0 for i in range(len(average_volume_list))])
    
    for ind in average_volume_list_index:
        if ind < 20:
            average_vol =  array_df[:,5][:ind].sum() / ind
        else:
            average_vol =  array_df[:,5][ind-20:ind].sum() / 20
        if array_df[:,5][ind] >= average_vol * 10:
            average_volume_test[ind] = 1
            
    return average_volume_test

#최종

In [ ]:
result = pd.DataFrame([], columns= ['shcode',"index"])
high_ratio = 0.15
volume_money = 30000000000

for i, v in enumerate(shcode_list['shcode']):
    
    now = datetime.now()
    now_time = '%s-%s-%s_%s:%s:%s' % ( now.year, now.month, now.day, now.hour, now.minute, now.second)
    print('------------------- test 시작_%s : , %s , %s , %s -------------------  ' %(now_time, i, v, shcode_list['shname'].iloc[i]))
    
    daily_data = query.query_ohlcv(v, to ='20190927')
    array_daily = np.array(daily_data)
    
    ratio_test = fun_incresing_ratio_test(array_daily, limit = high_ratio)
    if any(ratio_test) != 1:
        print('X_1. ratio_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))        
        continue
    print('O_1. ratio_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))  
    
    total_volume_test = fun_total_volume_test(array_daily, min_total_volume = volume_money)
    if any(total_volume_test) != 1:
        print('X_2. total_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))         
        continue
    print('O_2. total_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i])) 
    
    average_volume_test = fun_average_volume_test(array_daily, ratio_test, total_volume_test)
    if any(average_volume_test) != 1:
        print('X_3. average_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))            
        continue
    print('O_3. average_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))     
    
    result_test = ratio_test * total_volume_test * average_volume_test
    final_index = array_daily[:,0][result_test==1]
    final_value = " ".join(str(x) for x in final_index)
    #final_index = np.where(result_test==1)
    #final_value = " ".join(str(x) for x in final_index[0])
    
    test_result = pd.DataFrame([[v,final_value]], columns= ['shcode',"index"])
    result = result.append(test_result, ignore_index = True)

    print('------------------- test 완료 : , %s , %s , %s ------------------- ' %(i, v, shcode_list['shname'].iloc[i]))

filename = 'high_ratio'+'_'+str(high_ratio)+'_' + 'volume_money'+'_'+str(volume_money)+ '_v2' +'.csv'
result.to_csv(filename, sep=',')

In [6]:
result = pd.DataFrame([], columns= ['shcode',"index"])
high_ratio = 0.05
volume_money = 20000000000

for i, v in enumerate(shcode_list['shcode']):
    
    now = datetime.now()
    now_time = '%s-%s-%s_%s:%s:%s' % ( now.year, now.month, now.day, now.hour, now.minute, now.second)
    print('------------------- test 시작_%s : , %s , %s , %s -------------------  ' %(now_time, i, v, shcode_list['shname'].iloc[i]))
    
    daily_data = query.query_ohlcv(v, to ='20190927')
    array_daily = np.array(daily_data)
    
    ratio_test = fun_incresing_ratio_test(array_daily, limit = high_ratio)
    if any(ratio_test) != 1:
        print('X_1. ratio_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))        
        continue
    print('O_1. ratio_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))  
    
    total_volume_test = fun_total_volume_test(array_daily, min_total_volume = volume_money)
    if any(total_volume_test) != 1:
        print('X_2. total_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))         
        continue
    print('O_2. total_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i])) 
    
    average_volume_test = fun_average_volume_test(array_daily, ratio_test, total_volume_test)
    if any(average_volume_test) != 1:
        print('X_3. average_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))            
        continue
    print('O_3. average_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))     
    
    result_test = ratio_test * total_volume_test * average_volume_test
    final_index = array_daily[:,0][result_test==1]
    final_value = " ".join(str(x) for x in final_index)
    #final_index = np.where(result_test==1)
    #final_value = " ".join(str(x) for x in final_index[0])
    
    test_result = pd.DataFrame([[v,final_value]], columns= ['shcode',"index"])
    result = result.append(test_result, ignore_index = True)

    print('------------------- test 완료 : , %s , %s , %s ------------------- ' %(i, v, shcode_list['shname'].iloc[i]))

filename = 'high_ratio'+'_'+str(high_ratio)+'_' + 'volume_money'+'_'+str(volume_money)+ '_v2' +'.csv'
result.to_csv(filename, sep=',')

------------------- test 시작_2019-11-15_15:21:54 : , 0 , 000020 , 동화약품 -------------------  
O_1. ratio_test 통과 0 , 000020 , 동화약품
X_2. total_volume_test 없음 0 , 000020 , 동화약품
------------------- test 시작_2019-11-15_15:21:55 : , 1 , 000040 , KR모터스 -------------------  
X_1. ratio_test 없음 1 , 000040 , KR모터스
------------------- test 시작_2019-11-15_15:21:56 : , 2 , 000050 , 경방 -------------------  
X_1. ratio_test 없음 2 , 000050 , 경방
------------------- test 시작_2019-11-15_15:21:56 : , 3 , 000060 , 메리츠화재 -------------------  
X_1. ratio_test 없음 3 , 000060 , 메리츠화재
------------------- test 시작_2019-11-15_15:21:57 : , 4 , 000070 , 삼양홀딩스 -------------------  
X_1. ratio_test 없음 4 , 000070 , 삼양홀딩스
------------------- test 시작_2019-11-15_15:21:57 : , 5 , 000075 , 삼양홀딩스우 -------------------  
O_1. ratio_test 통과 5 , 000075 , 삼양홀딩스우
X_2. total_volume_test 없음 5 , 000075 , 삼양홀딩스우
------------------- test 시작_2019-11-15_15:21:57 : , 6 , 000080 , 하이트진로 -------------------  
O_1. ratio_test 통과 6 , 000080 , 하이트진로

O_1. ratio_test 통과 43 , 000670 , 영풍
O_2. total_volume_test 통과 43 , 000670 , 영풍
X_3. average_volume_test 없음 43 , 000670 , 영풍
------------------- test 시작_2019-11-15_15:22:14 : , 44 , 000680 , LS네트웍스 -------------------  
O_1. ratio_test 통과 44 , 000680 , LS네트웍스
X_2. total_volume_test 없음 44 , 000680 , LS네트웍스
------------------- test 시작_2019-11-15_15:22:15 : , 45 , 000700 , 유수홀딩스 -------------------  
O_1. ratio_test 통과 45 , 000700 , 유수홀딩스
X_2. total_volume_test 없음 45 , 000700 , 유수홀딩스
------------------- test 시작_2019-11-15_15:22:15 : , 46 , 000720 , 현대건설 -------------------  
O_1. ratio_test 통과 46 , 000720 , 현대건설
O_2. total_volume_test 통과 46 , 000720 , 현대건설
X_3. average_volume_test 없음 46 , 000720 , 현대건설
------------------- test 시작_2019-11-15_15:22:16 : , 47 , 000725 , 현대건설우 -------------------  
O_1. ratio_test 통과 47 , 000725 , 현대건설우
X_2. total_volume_test 없음 47 , 000725 , 현대건설우
------------------- test 시작_2019-11-15_15:22:16 : , 48 , 000760 , 이화산업 -------------------  
O_1. ratio_test 통과 4

O_1. ratio_test 통과 85 , 001390 , KG케미칼
X_2. total_volume_test 없음 85 , 001390 , KG케미칼
------------------- test 시작_2019-11-15_15:22:37 : , 86 , 001420 , 태원물산 -------------------  
O_1. ratio_test 통과 86 , 001420 , 태원물산
X_2. total_volume_test 없음 86 , 001420 , 태원물산
------------------- test 시작_2019-11-15_15:22:38 : , 87 , 001430 , 세아베스틸 -------------------  
O_1. ratio_test 통과 87 , 001430 , 세아베스틸
X_2. total_volume_test 없음 87 , 001430 , 세아베스틸
------------------- test 시작_2019-11-15_15:22:38 : , 88 , 001440 , 대한전선 -------------------  
O_1. ratio_test 통과 88 , 001440 , 대한전선
O_2. total_volume_test 통과 88 , 001440 , 대한전선
O_3. average_volume_test 통과 88 , 001440 , 대한전선
------------------- test 완료 : , 88 , 001440 , 대한전선 ------------------- 
------------------- test 시작_2019-11-15_15:22:39 : , 89 , 001450 , 현대해상 -------------------  
X_1. ratio_test 없음 89 , 001450 , 현대해상
------------------- test 시작_2019-11-15_15:22:39 : , 90 , 001460 , BYC -------------------  
O_1. ratio_test 통과 90 , 001460 , BYC
X_2. 

O_1. ratio_test 통과 127 , 002100 , 경농
O_2. total_volume_test 통과 127 , 002100 , 경농
O_3. average_volume_test 통과 127 , 002100 , 경농
------------------- test 완료 : , 127 , 002100 , 경농 ------------------- 
------------------- test 시작_2019-11-15_15:22:58 : , 128 , 002140 , 고려산업 -------------------  
O_1. ratio_test 통과 128 , 002140 , 고려산업
O_2. total_volume_test 통과 128 , 002140 , 고려산업
O_3. average_volume_test 통과 128 , 002140 , 고려산업
------------------- test 완료 : , 128 , 002140 , 고려산업 ------------------- 
------------------- test 시작_2019-11-15_15:22:59 : , 129 , 002150 , 도화엔지니어링 -------------------  
O_1. ratio_test 통과 129 , 002150 , 도화엔지니어링
O_2. total_volume_test 통과 129 , 002150 , 도화엔지니어링
O_3. average_volume_test 통과 129 , 002150 , 도화엔지니어링
------------------- test 완료 : , 129 , 002150 , 도화엔지니어링 ------------------- 
------------------- test 시작_2019-11-15_15:22:59 : , 130 , 002170 , 삼양통상 -------------------  
O_1. ratio_test 통과 130 , 002170 , 삼양통상
X_2. total_volume_test 없음 130 , 002170 , 삼양통상
--------

O_1. ratio_test 통과 165 , 002880 , 대유에이텍
O_2. total_volume_test 통과 165 , 002880 , 대유에이텍
X_3. average_volume_test 없음 165 , 002880 , 대유에이텍
------------------- test 시작_2019-11-15_15:23:18 : , 166 , 002900 , 동양물산 -------------------  
O_1. ratio_test 통과 166 , 002900 , 동양물산
O_2. total_volume_test 통과 166 , 002900 , 동양물산
X_3. average_volume_test 없음 166 , 002900 , 동양물산
------------------- test 시작_2019-11-15_15:23:19 : , 167 , 002920 , 유성기업 -------------------  
O_1. ratio_test 통과 167 , 002920 , 유성기업
X_2. total_volume_test 없음 167 , 002920 , 유성기업
------------------- test 시작_2019-11-15_15:23:19 : , 168 , 002960 , 한국쉘석유 -------------------  
O_1. ratio_test 통과 168 , 002960 , 한국쉘석유
X_2. total_volume_test 없음 168 , 002960 , 한국쉘석유
------------------- test 시작_2019-11-15_15:23:20 : , 169 , 002990 , 금호산업 -------------------  
O_1. ratio_test 통과 169 , 002990 , 금호산업
O_2. total_volume_test 통과 169 , 002990 , 금호산업
X_3. average_volume_test 없음 169 , 002990 , 금호산업
------------------- test 시작_2019-11-15_15:23:21 :

...

O_1. ratio_test 통과 2724 , 237880 , 클리오
O_2. total_volume_test 통과 2724 , 237880 , 클리오
X_3. average_volume_test 없음 2724 , 237880 , 클리오
------------------- test 시작_2019-11-15_15:43:25 : , 2725 , 238090 , 앤디포스 -------------------  
O_1. ratio_test 통과 2725 , 238090 , 앤디포스
O_2. total_volume_test 통과 2725 , 238090 , 앤디포스
X_3. average_volume_test 없음 2725 , 238090 , 앤디포스
------------------- test 시작_2019-11-15_15:43:25 : , 2726 , 238120 , 로고스바이오 -------------------  
O_1. ratio_test 통과 2726 , 238120 , 로고스바이오
X_2. total_volume_test 없음 2726 , 238120 , 로고스바이오
------------------- test 시작_2019-11-15_15:43:25 : , 2727 , 238200 , 비피도 -------------------  
O_1. ratio_test 통과 2727 , 238200 , 비피도
O_2. total_volume_test 통과 2727 , 238200 , 비피도
O_3. average_volume_test 통과 2727 , 238200 , 비피도
------------------- test 완료 : , 2727 , 238200 , 비피도 ------------------- 
------------------- test 시작_2019-11-15_15:43:26 : , 2728 , 238490 , 힘스 -------------------  
O_1. ratio_test 통과 2728 , 238490 , 힘스
O_2. total_volume

O_1. ratio_test 통과 2758 , 253840 , 수젠텍
O_2. total_volume_test 통과 2758 , 253840 , 수젠텍
O_3. average_volume_test 통과 2758 , 253840 , 수젠텍
------------------- test 완료 : , 2758 , 253840 , 수젠텍 ------------------- 
------------------- test 시작_2019-11-15_15:43:40 : , 2759 , 254120 , IBKS제5호스팩 -------------------  
O_1. ratio_test 통과 2759 , 254120 , IBKS제5호스팩
O_2. total_volume_test 통과 2759 , 254120 , IBKS제5호스팩
O_3. average_volume_test 통과 2759 , 254120 , IBKS제5호스팩
------------------- test 완료 : , 2759 , 254120 , IBKS제5호스팩 ------------------- 
------------------- test 시작_2019-11-15_15:43:40 : , 2760 , 255220 , SG -------------------  
O_1. ratio_test 통과 2760 , 255220 , SG
X_2. total_volume_test 없음 2760 , 255220 , SG
------------------- test 시작_2019-11-15_15:43:41 : , 2761 , 255440 , 야스 -------------------  
O_1. ratio_test 통과 2761 , 255440 , 야스
O_2. total_volume_test 통과 2761 , 255440 , 야스
X_3. average_volume_test 없음 2761 , 255440 , 야스
------------------- test 시작_2019-11-15_15:43:41 : , 2762 , 256150

X_1. ratio_test 없음 2792 , 264850 , IBKS제6호스팩
------------------- test 시작_2019-11-15_15:43:55 : , 2793 , 265480 , 미래에셋대우스팩1호 -------------------  
X_1. ratio_test 없음 2793 , 265480 , 미래에셋대우스팩1호
------------------- test 시작_2019-11-15_15:43:56 : , 2794 , 265520 , AP시스템 -------------------  
O_1. ratio_test 통과 2794 , 265520 , AP시스템
O_2. total_volume_test 통과 2794 , 265520 , AP시스템
X_3. average_volume_test 없음 2794 , 265520 , AP시스템
------------------- test 시작_2019-11-15_15:43:56 : , 2795 , 265560 , 영화테크 -------------------  
O_1. ratio_test 통과 2795 , 265560 , 영화테크
X_2. total_volume_test 없음 2795 , 265560 , 영화테크
------------------- test 시작_2019-11-15_15:43:57 : , 2796 , 265920 , 한화수성스팩 -------------------  
O_1. ratio_test 통과 2796 , 265920 , 한화수성스팩
O_2. total_volume_test 통과 2796 , 265920 , 한화수성스팩
X_3. average_volume_test 없음 2796 , 265920 , 한화수성스팩
------------------- test 시작_2019-11-15_15:43:57 : , 2797 , 267320 , 교보7호스팩 -------------------  
X_1. ratio_test 없음 2797 , 267320 , 교보7호스팩
-------------

O_1. ratio_test 통과 2830 , 290670 , 대보마그네틱
X_2. total_volume_test 없음 2830 , 290670 , 대보마그네틱
------------------- test 시작_2019-11-15_15:44:13 : , 2831 , 290720 , 푸드나무 -------------------  
O_1. ratio_test 통과 2831 , 290720 , 푸드나무
X_2. total_volume_test 없음 2831 , 290720 , 푸드나무
------------------- test 시작_2019-11-15_15:44:13 : , 2832 , 290740 , 액트로 -------------------  
O_1. ratio_test 통과 2832 , 290740 , 액트로
X_2. total_volume_test 없음 2832 , 290740 , 액트로
------------------- test 시작_2019-11-15_15:44:14 : , 2833 , 291210 , 한국제7호스팩 -------------------  
X_1. ratio_test 없음 2833 , 291210 , 한국제7호스팩
------------------- test 시작_2019-11-15_15:44:14 : , 2834 , 291230 , 삼성스팩2호 -------------------  
X_1. ratio_test 없음 2834 , 291230 , 삼성스팩2호
------------------- test 시작_2019-11-15_15:44:15 : , 2835 , 293580 , 나우아이비캐피탈 -------------------  
O_1. ratio_test 통과 2835 , 293580 , 나우아이비캐피탈
X_2. total_volume_test 없음 2835 , 293580 , 나우아이비캐피탈
------------------- test 시작_2019-11-15_15:44:15 : , 2836 , 293780 , 압타바이오 

O_1. ratio_test 통과 2871 , 317830 , 에스피시스템스
O_2. total_volume_test 통과 2871 , 317830 , 에스피시스템스
X_3. average_volume_test 없음 2871 , 317830 , 에스피시스템스
------------------- test 시작_2019-11-15_15:44:31 : , 2872 , 317850 , 대모 -------------------  
O_1. ratio_test 통과 2872 , 317850 , 대모
O_2. total_volume_test 통과 2872 , 317850 , 대모
O_3. average_volume_test 통과 2872 , 317850 , 대모
------------------- test 완료 : , 2872 , 317850 , 대모 ------------------- 
------------------- test 시작_2019-11-15_15:44:32 : , 2873 , 318000 , 한국바이오젠 -------------------  
O_1. ratio_test 통과 2873 , 318000 , 한국바이오젠
O_2. total_volume_test 통과 2873 , 318000 , 한국바이오젠
X_3. average_volume_test 없음 2873 , 318000 , 한국바이오젠
------------------- test 시작_2019-11-15_15:44:32 : , 2874 , 319400 , 엔에이치스팩14호 -------------------  
X_1. ratio_test 없음 2874 , 319400 , 엔에이치스팩14호
------------------- test 시작_2019-11-15_15:44:33 : , 2875 , 319660 , 피에스케이 -------------------  
O_1. ratio_test 통과 2875 , 319660 , 피에스케이
O_2. total_volume_test 통과 2875 , 319660